## Segmenting and Clustering Neighborhoods in Toronto

### Date: 24 March 2019
### Author: Min Jung Kang

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml

In [2]:
from bs4 import BeautifulSoup

#### Part 1. Scraping Wikipedia page data into a dataframe

In [3]:
# Obtaining data from Wikipedia
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [4]:
PostalCode = []
Borough = []
Neighborhood = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if len(cells) > 0:
        PostalCode.append(cells[0].text)
        Borough.append(cells[1].text)
        Neighborhood.append(cells[2].text.rstrip('\n'))

In [5]:
dict = { 'PostalCode' : PostalCode,
       'Borough' : Borough,
       'Neighborhood': Neighborhood}
df = pd.DataFrame.from_dict(dict)
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
# Remove 'Not assigned' Boroughs
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [7]:
# Copy Borough values to 'Not assigned' Neighborhoods
df.replace("Not assigned", np.nan, inplace=True)
df['Neighborhood'].fillna(df['Borough'], inplace=True)
df.head(20)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [8]:
# Group by PostalCode
df = df.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
df.shape

(103, 3)

#### Part 2. Adding coordinates to the dataframe

In [10]:
geodata = pd.read_csv('../projects/Geospatial_Coordinates.csv')
geodata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
# Rename column in geodata to merge with df
geodata.rename(columns={"Postal Code" : "PostalCode"}, inplace=True)
geodata.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# Outer join df and geodata
coords = pd.merge(df, geodata, on = 'PostalCode', how = 'outer')

In [13]:
coords.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### Part 3. Exploring the neighborhood

In [14]:
import folium
from geopy.geocoders import Nominatim

In [15]:
# get geographical coordinates for Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [16]:
# make a map of Toronto using folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(coords['Latitude'], coords['Longitude'], coords['Borough'], coords['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
# Define Foursquare credentials and version
CLIENT_ID = 'I2R3YKTK5MX53AY50035ANYKRHGRDWTFCWEAIGR2BFKMNNPK'
CLIENT_SECRET = '1UIH5XCEZJR3KIOTB1QH4NRKX2KQJLSUQLN54EGQHUZ02SY2'
VERSION = '20190324'

In [18]:
# Picking a random neighborhood to explore
neighborhood_latitude = coords.loc[3, 'Latitude']
neighborhood_longitude = coords.loc[3, 'Longitude']
neighborhood_name = coords.loc[3, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Woburn are 43.7709921, -79.21691740000001.


In [19]:
# Top 10 venues in Woburn within 500 meters radius
LIMIT = 10
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c96cc8ff594df6afaca02bf'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 67,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          

In [20]:
from pandas.io.json import json_normalize

In [21]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean column names
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Textile Museum of Canada,Art Museum,43.654396,-79.386500
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Cafe Plenty,Café,43.654571,-79.389450
5,Manpuku まんぷく,Japanese Restaurant,43.653612,-79.390613
6,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684
7,Tsujiri,Tea Room,43.655374,-79.385354
8,Poke Guys,Poke Place,43.654895,-79.385052
9,Nathan Phillips Square,Plaza,43.652270,-79.383516


In [22]:
#Create a function to repeat the process to all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=coords['Neighborhood'],
                                   latitudes=coords['Latitude'],
                                   longitudes=coords['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West,Steeles West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beac

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(693, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [25]:
count = toronto_venues.groupby('Neighborhood').count()
count = count[['Venue']]
count.rename = {'Total Venues' : 'Venue'}
count.head()

,Venue
Neighborhood,
"Adelaide,King,Richmond",10
Agincourt,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10
"Alderwood,Long Branch",8


In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 179 uniques categories.


In [36]:
# One hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe and bring to front
neighbor = toronto_venues['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neighbor)

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_onehot.shape

(693, 179)

In [38]:
# Group rows by neighborhoodand take the mean of the frequency of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Art Gallery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0
1,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0
4,"Alderwood,Long Branch",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
5,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
6,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
7,"Bedford Park,Lawrence Manor East",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
8,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9,"Birch Cliff,Cliffside West",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [39]:
toronto_grouped.shape

(99, 179)

In [40]:
# top 5 venues for each neighborhood
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0        Steakhouse   0.2
1  Greek Restaurant   0.1
2  Asian Restaurant   0.1
3             Hotel   0.1
4      Concert Hall   0.1


----Agincourt----
                venue  freq
0              Lounge   0.2
1      Breakfast Spot   0.2
2        Skating Rink   0.2
3  Chinese Restaurant   0.2
4      Sandwich Place   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0                Park  0.67
1          Playground  0.33
2  Miscellaneous Shop  0.00
3              Lounge  0.00
4              Market  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                 venue  freq
0        Grocery Store   0.2
1  Fried Chicken Joint   0.1
2          Coffee Shop   0.1
3       Sandwich Place   0.1
4             Pharmacy   0.1


----Alderwood,Long Branch----
          venue  freq
0   Pizza Place  0.25
1   Coffee Shop  0.12
2  Skati

                   venue  freq
0          Grocery Store  0.25
1                   Park  0.25
2                   Bank  0.25
3                  Hotel  0.25
4  Performing Arts Venue  0.00


----East Birchmount Park,Ionview,Kennedy Park----
                venue  freq
0         Bus Station  0.14
1         Coffee Shop  0.14
2   Convenience Store  0.14
3          Hobby Shop  0.14
4  Chinese Restaurant  0.14


----East Toronto----
               venue  freq
0        Coffee Shop  0.33
1               Park  0.33
2  Convenience Store  0.33
3              Motel  0.00
4     Massage Studio  0.00


----Emery,Humberlea----
               venue  freq
0     Baseball Field   1.0
1  Accessories Store   0.0
2  Mobile Phone Shop   0.0
3             Market   0.0
4     Massage Studio   0.0


----Fairview,Henry Farm,Oriole----
                 venue  freq
0             Pharmacy   0.1
1    Electronics Store   0.1
2         Burger Joint   0.1
3        Shopping Mall   0.1
4  American Restaurant   0.1


----Firs

               venue  freq
0        Coffee Shop  0.25
1                Pub  0.25
2  Health Food Store  0.25
3              Motel  0.00
4     Massage Studio  0.00


----The Beaches West,India Bazaar----
               venue  freq
0       Liquor Store   0.1
1     Ice Cream Shop   0.1
2         Steakhouse   0.1
3       Burger Joint   0.1
4  Fish & Chips Shop   0.1


----The Danforth West,Riverdale----
                venue  freq
0    Greek Restaurant   0.4
1  Italian Restaurant   0.2
2             Brewery   0.1
3      Ice Cream Shop   0.1
4      Cosmetics Shop   0.1


----The Junction North,Runnymede----
                  venue  freq
0               Brewery  0.33
1  Caribbean Restaurant  0.33
2              Bus Line  0.33
3     Accessories Store  0.00
4                 Motel  0.00


----The Kingsway,Montgomery Road,Old Mill North----
                venue  freq
0                Park  0.33
1          Smoke Shop  0.33
2               River  0.33
3   Accessories Store  0.00
4  Miscellaneous 

In [42]:
# Put into pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Steakhouse,Coffee Shop,Greek Restaurant,Speakeasy,Vegetarian / Vegan Restaurant,Concert Hall,Plaza,Asian Restaurant,Hotel,Yoga Studio
1,Agincourt,Lounge,Chinese Restaurant,Skating Rink,Breakfast Spot,Sandwich Place,Yoga Studio,Curling Ice,Dim Sum Restaurant,Dessert Shop,Department Store
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Yoga Studio,Cuban Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pharmacy,Fried Chicken Joint,Pizza Place,Sandwich Place,Beer Store,Fast Food Restaurant,Coffee Shop,Video Store,French Restaurant
4,"Alderwood,Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pharmacy,Gym,Sandwich Place,Pub,Creperie,Dessert Shop,Department Store


In [47]:
# K Means Clustering
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [44]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0])

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# merge toronto_grouped with toronto coords to add latitude/longitude for each neighborhood
toronto_merged = coords
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2.0,Bar,Yoga Studio,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0.0,Spa,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Pizza Place,Construction & Landscaping,Convenience Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Dance Studio,Dog Run,Discount Store


In [56]:
toronto_merged.shape

(103, 16)

In [59]:
toronto_merged.dropna(inplace=True)
toronto_merged.shape

(99, 16)

In [65]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2,Bar,Yoga Studio,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,0,Spa,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Pizza Place,Construction & Landscaping,Convenience Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Dance Studio,Dog Run,Discount Store


In [66]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
# Cluster 0
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0,Spa,Rental Car Location,Breakfast Spot,Intersection,Electronics Store,Medical Center,Mexican Restaurant,Pizza Place,Construction & Landscaping,Convenience Store
3,Scarborough,0,Coffee Shop,Korean Restaurant,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
4,Scarborough,0,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Bank,Bakery,Athletics & Sports,Dance Studio,Dog Run,Discount Store
6,Scarborough,0,Coffee Shop,Bus Station,Discount Store,Chinese Restaurant,Department Store,Convenience Store,Hobby Shop,Diner,Dim Sum Restaurant,Dessert Shop
7,Scarborough,0,Bus Line,Bakery,Metro Station,Park,Intersection,Fast Food Restaurant,Bus Station,Soccer Field,Creperie,Cuban Restaurant
8,Scarborough,0,Movie Theater,American Restaurant,Motel,Yoga Studio,Curling Ice,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
9,Scarborough,0,General Entertainment,College Stadium,Skating Rink,Café,Curling Ice,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
10,Scarborough,0,Indian Restaurant,Pet Store,Furniture / Home Store,Thrift / Vintage Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Gaming Cafe,Curling Ice,Diner
11,Scarborough,0,Middle Eastern Restaurant,Auto Garage,Breakfast Spot,Shopping Mall,Sandwich Place,Yoga Studio,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
12,Scarborough,0,Lounge,Chinese Restaurant,Skating Rink,Breakfast Spot,Sandwich Place,Yoga Studio,Curling Ice,Dim Sum Restaurant,Dessert Shop,Department Store


In [69]:
# Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,1,Pizza Place,Playground,Yoga Studio,Cuban Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
100,Etobicoke,1,Pizza Place,Park,Mobile Phone Shop,Creperie,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega,Dance Studio


In [70]:
# Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,2,Bar,Yoga Studio,Dance Studio,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store


In [71]:
# Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega


In [72]:
# Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,4,Park,Playground,Yoga Studio,Cuban Restaurant,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
23,North York,4,Park,Bank,Yoga Studio,Curling Ice,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
25,North York,4,Bus Stop,Park,Fast Food Restaurant,Food & Drink Shop,Cuban Restaurant,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
30,North York,4,Bus Stop,Airport,Park,Curling Ice,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
31,North York,4,Park,Grocery Store,Bank,Hotel,Yoga Studio,Dance Studio,Dog Run,Discount Store,Diner,Dim Sum Restaurant
40,East York,4,Park,Convenience Store,Coffee Shop,Yoga Studio,Dance Studio,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
44,Central Toronto,4,Bus Line,Park,Swim School,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
50,Downtown Toronto,4,Park,Playground,Trail,Yoga Studio,Creperie,Diner,Dim Sum Restaurant,Dessert Shop,Department Store,Deli / Bodega
64,Central Toronto,4,Park,Sushi Restaurant,Trail,Jewelry Store,Curling Ice,Dog Run,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
73,York,4,Hockey Arena,Park,Trail,Field,Yoga Studio,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop,Department Store
